# Haunted Mansion Project

Welcome to the Haunted Mansion Project! This project aims to create an immersive and interactive experience that simulates a haunted mansion. The project covers the following aspects:

1. **Storyline Development**: Crafting a compelling and spooky storyline to engage the audience.
2. **Character Design**: Creating detailed and eerie characters that enhance the haunted atmosphere.
3. **Environment Creation**: Designing a haunted mansion environment with realistic and chilling elements.
4. **Sound Design**: Incorporating creepy sound effects and music to heighten the suspense and fear.
5. **Interactive Elements**: Adding interactive features that allow users to explore and interact with the haunted mansion.
6. **User Experience**: Ensuring a seamless and immersive experience for the users.

By the end of this project, you will have a fully developed haunted mansion experience that can be enjoyed by users. Let's dive into the world of ghosts and ghouls!

In [1]:
import time

In [2]:
# All rooms, items and doors for game configuration


"""MB: Need to adjust layout (rooms, items and doors[keys for doors]) to drawio -> GF"""

rooms = {
    'Foyer': {
        'description': 'A dimly lit entrance hall with a grand staircase.',
        'items': ['Smelly Key']
    },
    'Library': {
        'description': 'Walls lined with ancient books. A cold draft chills you.',
        'items': ['Silver Key']
    },
    'Dining Room': {
        'description': 'An elegant dining room with an old chandelier.',
        'items': ['Bloody Hammer']
    },
    'Kitchen': {
        'description': 'A dark kitchen filled with strange smells.',
        'items': ['Wooden Box', 'Flashlight']                                     # GFM: I would delete de Flashlight or leave it as an item that doesn't do anything?
    },                              # GFM: The wooden box has to also be defined as somthing that contains the "Main Door Key"
    'Exit':{
        "description": "You made it",
        "items": []
    }
}

keys = {
    "Smelly Key": {         #GFM added keys             GFM Wed changed key description
        "name": "Smelly Key",
        "description": "A rusted key reeking of formaldehyde and rot. The stench is almost unbearable, quite disgusting.The engraving reads: LABORATORY ACCESS." #GFM 2 
    },
    "Silver Key": {
        "name": "Silver Key",
        "description": "A tarnished silver key, icy to the touch. Etched into it are the words: LIBRARY ARCHIVES."
    },
    "Main Door Key": {
        "name": "Iron Key",
        "description": "A massive iron key, heavy and cold. Its teeth are shaped like screaming faces."
    }
}

items = {
    "Bloody Hammer": {
        "name": "Bloody Hammer",
        "description": "It's a bloody hammer!"
    },
    "Wooden Box": {
        "name": "Wooden Box",
        "description": "Some sort of wooden box."
    },
    "Flashlight": {
        "name": "Flashlight",
        "description": "Makes light"
    }
}
 # GFM 2 <-- Added key references to doors
doors = {
     "Squeaky Door": {
    "name":"Squeaky Door",
    "description": "Makes a lot of noise",
    "connections": ["Library", "Dining Room"],
    "locked": True,
    "key": "Silver Key"                            # GFM 2 Wed <-- Added key reference
    },
    "Wooden Door": {
    "name":"Wooden Door",
    "description": "A cracked wooden door. A foul, rotting stench seeps through the gaps.",
    "connections": ["Kitchen", "Dining Room"],
    "locked": True,
    "key": "Smelly Key"                            # GFM 2 Wed <-- Added key reference
    },
    "Bloody Door": {
    "name":"Bloody Door",
    "description": "A fancy door stained with dried blood, Pollock style!",
    "connections": ["Foyer", "Dining Room"],
    "locked": False,
    "key": None  
    },
    "Main Iron Door": {
    "name":"Main Iron Door",
    "description": "Big double iron door",
    "connections": ["Exit", "Foyer"],
    "locked": True,
    "key": "Main Door Key"                            # GFM 2 Wed <-- Added key reference
    },
}



In [3]:
# classes initialisation

class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.connections = []
        self.items = []

    def describe_room(self):
        if self.items:
            print(f"The following item(s) are in the room:")
            for item in self.items:
                print("- "+str(item))
        print("To interact with the above, type their name")
        return

    def __str__(self):
        return f"Room Object: {self.name}"
    
    def __repr__(self):
        return f"Room(name={self.name!r})"

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

    def inspect(self):
        # need to make unique cases for special items
        print(self.description)
        return self

    def __str__(self):
        return f"{self.name}"

    def __repr__(self):
        return f"Item(name={self.name!r})"

class Door(Item):
    def __init__(self, name, description, room1, room2, locked, key):                     # New argument key   # GFM WEd. Modify the Door class to include key information
        super().__init__(name, description)
        self.room1 = room1
        self.room2 = room2
        self.locked = locked
        self.key = key                           # New property               # GFM WEd. Modify the Door class to include key information

    def __repr__(self):
        return f"Door(name={self.name!r})"
    
class Key(Item):
    def __init__(self, name, description):
        super().__init__(name, description)

class Furniture(Item):
    def __init__(self, name, description):
        super().__init__(name, description)
    
class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []
        self.escaped = False
        self.name = None
        self.moves = 0

    def move(self, door):
        # check the connections of the door and adjust the location to the room wich is not the current one
        if self.current_room == door.room1:
            self.current_room = door.room2
        else:
            self.current_room = door.room1
        print(f"{self.current_room.description}. This must be the {self.current_room.name}.")

    def show_inventory(self):
        print("The following items are in your inventory:")
        for item in self.inventory:
            print(str(item))


In [4]:
def timing(start_time, timer_duration):
    current_time = time.time()
    elapsed_time = current_time - start_time
    time_left = timer_duration - elapsed_time
    minutes = int(time_left // 60)
    remaining_seconds = int(time_left % 60)
    print(f"You have {minutes} minutes and {remaining_seconds} seconds left")

def gameover():
    print("Time's up! The spirits of the haunted mansion have trapped you forever!")
    print("""
 _____                           _____                   
|  __ \                         |  _  |                  
| |  \/  __ _  _ __ ___    ___  | | | |__   __ ___  _ __ 
| | __  / _` || '_ ` _ \  / _ \ | | | |\ \ / // _ \| '__|
| |_\ \| (_| || | | | | ||  __/ \ \_/ / \ V /|  __/| |   
 \____/ \__,_||_| |_| |_| \___|  \___/   \_/  \___||_|                                                      
""")

# Add this function at the end
def display_outro():
    print("""
    \033[1mYou burst through the Main Iron Door into the moonlight!\033[0m
    
    The cold night air feels like freedom itself. As the door slams shut behind you, 
    the eerie wails of the mansion fade into the distance. You've escaped the Eclipsed Manor, 
    but its secrets will haunt your dreams forever... 
    
    \033[3m(Thank you for playing!)\033[0m
    """)    

In [5]:
rooms_objects = {}
items_objects = {}

# dictionary of rooms to store Room instances of each room
for room in rooms:
    rooms_objects[room] = Room(room, rooms[room]["description"])

# dictionary of items to store Items instances of each item
for item in items:
    if item in keys:  # If it's a key, initialize as Key
        items_objects[item] = Key(item, items[item]["description"])
    else:  # Otherwise, initialize as Item
        items_objects[item] = Item(item, items[item]["description"])
# for item in items:                                                    # Try like this for the hammer!
#     items_objects[item] = Furniture(item, items[item]["description"])

# dictionary of keys to store Key instances of each key
for key_name in keys:
    key_data = keys[key_name]
    items_objects[key_name] = Key(
        name=key_name,
        description=key_data["description"],
    )

# add items to rooms
for room in rooms:
    for item in rooms[room]["items"]:
        rooms_objects[room].items.append(items_objects[item])

# dictionary of doors to store Door instance of each door
# GFM changed it for better reading 

for door_name in doors:
    door_data = doors[door_name]
    items_objects[door_name] = Door(
        name=door_name,
        description=door_data["description"],
        room1=rooms_objects[door_data["connections"][0]],
        room2=rooms_objects[door_data["connections"][1]],
        locked=door_data["locked"],
        key=door_data.get("key", None)          # ensures that if a door doesn't have a "key" property, it defaults to None
    )

# add room connections to rooms based on doors connections (later used to verify play move input)
for door in filter(lambda x: isinstance(items_objects[x], Door), items_objects):
    room1 = items_objects[door].room1
    room2 = items_objects[door].room2
    room1.connections.append(room2)
    room2.connections.append(room1)

    # add doors as items to rooms
    room1.items.append(items_objects[door])
    room2.items.append(items_objects[door])

# create player and set starting room
player = Player()
player.current_room = rooms_objects["Dining Room"]    # GFM -> change to start in Library

In [6]:
rooms_objects["Kitchen"].items

[Item(name='Wooden Box'), Item(name='Flashlight'), Door(name='Wooden Door')]

In [7]:
timer_duration = 60 * 10 # 10 minutes
start_time = time.time()


In [ ]:
# Introduce the game
print("Welcome. This is what you have to do. Everything written in '' can be interacted with.\nYou can end your life with 'quit' and end this nightmare.")
# name = input("What is your name?")
# player.name = name
# print(f"Hello {player.name}")

# Start the game
# while not exited:
while not player.escaped:
    # timer for the game
    if time.time() > start_time + timer_duration:
        gameover()
        break
    player.moves += 1 # move counter
    answer = input("What should i do? 'scout' the room or 'inspect' something?")             #GFM changed 
    if answer == "quit":
        break
    # look around
    elif answer == "scout":
        player.current_room.describe_room()

    # Item interaction
    elif answer in items_objects:
        item = items_objects[answer]

        # Check if the item is in the current room
        if item not in player.current_room.items:
            print(f"There is no {item.name} here.")
            continue            # Skip to the next iteration

        # Handle Wooden Box interaction
        if item.name == "Wooden Box":
            if any(inv_item.name == "Bloody Hammer" for inv_item in player.inventory):
                print("You smash the wooden box open with the hammer!")
                print("Inside, you find the Main Door Key!")

                 # Add Iron Key to the room
                iron_key = items_objects["Main Door Key"]
                player.current_room.items.append(iron_key)

                # Remove the wooden box
                player.current_room.items.remove(item)
            
                # Prompt the player to pick up the key
                pickup = input("Do you want to pick up the Main Door Key? (Y/N) ").upper()
                if pickup == "Y":
                    player.inventory.append(iron_key)
                    player.current_room.items.remove(iron_key)
                    print(f"{iron_key.name} added to inventory!")
            else:
                print("The wooden box is too sturdy to open without a tool.")
            continue

        # Handle doors
        if isinstance(item, Door):
            # GFM: Check if the player is exiting through the Main Iron Door
            if item.name == "Main Iron Door" and player.current_room.name == "Exit":
                display_outro()
                player.escaped = True
                break
                
            # Handle locked doors
            elif item.locked:
                # check if player has the key
                has_key = any(isinstance(inv_item, Key) and inv_item.name == item.key for inv_item in player.inventory)
                if has_key:

                    item.locked = False
                    print(f"You unlocked the {item.name} with the {item.key}!")
                    player.move(item)
                else:
                    print(f"The {item.name} is locked! You need the {item.key}.")

    # Handle unlocked doors            
            else:
                player.move(item)
                

        # Handle keys and other items
        elif isinstance(item, Key) or item.name == "Bloody Hammer": #or isinstance(item, Furniture):
            item.inspect()
            pickup = input("Do you want to pick up this item? ('Y' | 'N') ").upper()
            if pickup == "Y":
                player.inventory.append(item)
                player.current_room.items.remove(item)
                print(f"{item.name} added to inventory")
    
    elif answer == "inventory":
        player.show_inventory()

    elif answer == "time":
        timing(start_time, timer_duration)

    else:
        print("Invalid command. Try 'scout', 'inspect', or a door/item name.")


if player.escaped:
    print(f"Congrats you escaped with {player.moves} commands")
else:
    gameover()

Welcome. This is what you have to do. Everything written in '' can be interacted with.
You can end your life with 'quit' and end this nightmare.
The following item(s) are in the room:
- Bloody Hammer
- Squeaky Door
- Wooden Door
- Bloody Door
To interact with the above, type their name
It's a bloody hammer!
Bloody Hammer added to inventory
The Wooden Door is locked! You need the Smelly Key.
A dimly lit entrance hall with a grand staircase.. This must be the Foyer.
A rusted key reeking of formaldehyde and rot. The stench is almost unbearable, quite disgusting.The engraving reads: LABORATORY ACCESS.
Smelly Key added to inventory
There is no Wooden Door here.
An elegant dining room with an old chandelier.. This must be the Dining Room.
The following item(s) are in the room:
- Squeaky Door
- Wooden Door
- Bloody Door
To interact with the above, type their name
You unlocked the Wooden Door with the Smelly Key!
A dark kitchen filled with strange smells.. This must be the Kitchen.
The followi

In [9]:
# implemented move counter
# implemented a timer
# added gameover
# example code formatting below

# print("\033[1mThis is bold text\033[0m")
# print("This is normal text")
# print("\033[3mThis is italic text\033[0m")
# print("This is \033[3mitalic\033[0m and this is normal.")

In [10]:
# input("""You can do the following things
#     1. \033[1mscout\033[0m to scan the room
#     2. \033[1minspect\033[0m to inspect an item\n""")

In [11]:
# GFM changed starts:

# Define the display_intro function
def display_intro():
        # Typographic ASCII Logo for "Escape from Eclipsed Manor"
    logo = r"""Logo
    """
# ASCII Art of a Bookshelf
    bookshelf = r"""
       ASCII Art
    """

    # Intro Text
    intro_text = """
    Text 1
    """
    # ASCII Art of a Bookshelf
    bookshelf = r"""
    ASCII Art       
    """

    intro_text2 = """ Text 2
    """

    # Display the ASCII art and intro text
    print(logo)
    print(intro_text)
    print(bookshelf)
    print(intro_text2)

In [12]:
player.current_room.items

[Door(name='Main Iron Door')]

GFM Notes from 26.02.25 23:30

2dos: 
- list all commands available to options
- "hello world".title() to capitalize the first letter of each word in a string
- Function def display_outro() at the end when you get out of the house
- a Where am i? function to tell which room is it there and maybe give again the description?
- Where is the Main Door Key?

/ I added the hammer & box function but still i cannot pick up the main key